In [1]:
from IPython.display import display, HTML
import re
import requests
import json
import numpy as np
import pandas as pd
import urllib.parse

In [2]:
keyword = input()
encoded = urllib.parse.quote(keyword)
data_sp = []

headers = {
    'referer': 'https://tiki.vn/',
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36 Edg/140.0.0.0',
    'cookie' : '_trackity=06861abb-5fa7-dbfb-e863-8ce9c5b0178b; _ga=GA1.1.296953344.1758019280; _gcl_au=1.1.1441505066.1758019284; _fbp=fb.1.1758019283781.249312387863426626; _tt_enable_cookie=1; _ttp=01K591TV5X01A4KHKCDTKKMWKG_.tt.1; __iid=749; __iid=749; __su=0; __su=0; __RC=4; __R=1; _hjSessionUser_522327=eyJpZCI6IjVhNmJmMTIxLWY3MWQtNTI3Mi04ZmYwLWRlNjQyYWQ3MDJjOSIsImNyZWF0ZWQiOjE3NTgwMTkyODQyMDIsImV4aXN0aW5nIjp0cnVlfQ==; __tb=0; __utm=source%3Daccesstrade%7Cmedium%3Dtiki-aff%7Ccampaign%3DAFF_NBR_ACT_UNK_TIKIVN-TNWGVSKG_ALL_VN_ALL_UNK_UNK_TAPX.b9c70e87-a6cf-4614-8d2f-52ad0f233df4_TAPU.7e816ab8-b482-4a28-bd38-1b2a6d1159b8; __utm=source%3Daccesstrade%7Cmedium%3Dtiki-aff%7Ccampaign%3DAFF_NBR_ACT_UNK_TIKIVN-TNWGVSKG_ALL_VN_ALL_UNK_UNK_TAPX.b9c70e87-a6cf-4614-8d2f-52ad0f233df4_TAPU.7e816ab8-b482-4a28-bd38-1b2a6d1159b8; G_ENABLED_IDPS=google; TOKENS={%22access_token%22:%22f7a0tYUhVgFXwW4ymxC9Aqv8ipdBnKH2%22}; cto_bundle=zZ26Tl83MHJRZ2NrYmFTM3F2WmE5c3RoNDdLZUZyYXFqV0clMkZEM3FQMGxzdER5RmRHallaMGhicTJnWk1kSHRLNm1nVWlqJTJCeFpWZmpKNVM5S0ZpUmFCS1dYaFNqJTJCVUwxWVdsRzA4YnJoN1diYUxXTGpOdnpyYkFCU3NtMTFwQ0xrMDBGczE5YUpablJobyUyRlE0TVVZaHF3UDU5QSUzRCUzRA; delivery_zone=Vk4wMzQwMjQwMTM=; tiki_client_id=296953344.1758019280; _hjSession_522327=eyJpZCI6IjVjZjY5MjZlLTE5NDAtNDkyNS1hMjYzLWY2NTE4MDZkNmNiYyIsImMiOjE3NjEwMTQ5MTcwNjIsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjowLCJzcCI6MH0=; __IP=0; ttcsid=1761014916995::iDzJNBQU-UZnsGv6HuWV.13.1761015165883.0; ttcsid_D031T03C77UFNM54TPR0=1761014916994::X0FLCxHnYFOWLYAR8VTh.13.1761015165883.0; __uif=__uid%3A1928707292284752859%7C__ui%3A-1%7C__create%3A1759720795; _ga_S9GLR1RQFJ=GS2.1.s1761014911$o16$g1$t1761015246$j60$l0$h0; amp_99d374=iX38rQOfpHFuKV8zhhl8QV.MzA1MDY0ODk=..1j82am37s.1j82b0cu9.a2.dd.nf'
}

for i in range(1, 6):
    url = f'https://tiki.vn/api/v2/products?limit=82&include=advertisement&aggregations=2&trackity_id=06861abb-5fa7-dbfb-e863-8ce9c5b0178b&q={encoded}&page={i}'
    response = requests.get(url, headers=headers)
    data_json = response.json()
    data_sp += data_json.get('data', [])


In [ ]:
data_sp

In [ ]:
response = requests.get(url, headers=headers)
response.status_code

In [3]:
dataframe = {
    'Source_productID': [],
    'ProductName': [],
    'Price': [],
    'Original_Price': [],
    'Discount': [],
    'Quantity_Sold': [],
    'Rating': [],
    'Review_Count': [],
    'URL_Product': [],
    'URL_Image': [],
    'Source': []
}

for sp in data_sp:
    dataframe['Source_productID'].append(int(sp['id']))
    dataframe['ProductName'].append(sp['name'])
    dataframe['Source'].append('Tiki')
    dataframe['Price'].append(float(sp['price']))
    dataframe['Original_Price'].append(float(sp['original_price']) if float(sp['original_price']) > 0 else "")
    dataframe['Quantity_Sold'].append(int(sp.get('quantity_sold', {'value': 0}).get('value', 0)))
    dataframe['Discount'].append(float(sp['discount_rate']))
    dataframe['Rating'].append(float(sp['rating_average']) if int(sp['rating_average']) > 0 else None)
    dataframe['Review_Count'].append(int(sp['review_count']))
    if 'advertisement' in sp and 'ad' in sp['advertisement'] and len(sp['advertisement']['ad']) > 0:
        url = sp['advertisement']['ad'][0]['properties'].get('url', np.nan)
    else:
        url = np.nan
    dataframe['URL_Product'].append(url)
    dataframe['URL_Image'].append(sp['thumbnail_url'])

# dataframe['discount'] = [f"{d}%" for d in dataframe['discount']]
df = pd.DataFrame(dataframe)

# df['url_product'] = df['url_product'].apply(lambda x: f'<a href="{x}" target="_blank">Link</a>')
# df['url_image'] = df['url_image'].apply(lambda x: f'<a href="{x}" target="_blank">Link</a>')

# display(HTML(df.to_html(escape=False)))



In [4]:
df

,Source_productID,ProductName,Price,Original_Price,Discount,Quantity_Sold,Rating,Review_Count,URL_Product,URL_Image,Source
0,95295983,"SET 3 Bút Chì, Gôm Và Đệm Cầm Bút PLUS",30600.0,34000.0,10.0,10,5.0,3,https://tiki.vn/set-3-but-chi-gom-va-dem-cam-b...,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tiki
1,95295984,Combo 12 Cây Bút Dạ Quang Xóa Được PLUS,217800.0,250000.0,13.0,8,5.0,1,https://tiki.vn/combo-12-cay-but-da-quang-xoa-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tiki
2,91457818,Bút viết máy cao cấp mực đều đẹp màu Xanh đen/...,110000.0,230000.0,52.0,132,4.5,33,https://tiki.vn/but-viet-may-cao-cap-muc-deu-d...,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tiki
3,185408437,Bút Bi Nước Mực Gel Bấm Ngòi 0.5mm Deli - Mực ...,9000.0,19800.0,55.0,1644,4.8,84,https://tiki.vn/but-bi-nuoc-muc-gel-bam-ngoi-0...,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tiki
4,253136515,Bút chì 2B Deli7084 (12 cây) - Hàng Chính Hãng,62000.0,85000.0,27.0,52,5.0,4,NaN,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tiki
...,...,...,...,...,...,...,...,...,...,...,...
235,23407657,Bộ 2 Vỉ 3 Bút chì khúc PC-09 (Sản Phẩm Nhiều M...,16600.0,18500.0,10.0,23,NaN,0,NaN,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tiki
236,278455163,Bút Chì Gỗ Đen 2B Có Tẩy - Hồng Hà 3587,4700.0,5000.0,6.0,0,NaN,0,NaN,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tiki
237,120472286,Bút Chì Mỹ Thuật 4B Thiên Long GP-023,8300.0,8500.0,2.0,17,5.0,1,NaN,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tiki
238,34773583,Bút chì gỗ STABILO PC282-4B-Othello graphic pe...,29100.0,30000.0,3.0,28,4.0,1,NaN,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tiki


In [5]:
# df.replace('', np.nan, inplace=True)
df.dropna(subset=['URL_Product'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
df.drop_duplicates(subset=['Source_productID'], keep='first', inplace=True)


In [9]:
df

,Source_productID,ProductName,Price,Original_Price,Discount,Quantity_Sold,Rating,Review_Count,URL_Product,URL_Image,Source
0,95295983,"SET 3 Bút Chì, Gôm Và Đệm Cầm Bút PLUS",30600.0,34000.0,10.0,10,5.0,3,https://tiki.vn/set-3-but-chi-gom-va-dem-cam-b...,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tiki
5,276047906,Bút lông tô màu Crylic Marker 60 màu cao cấp v...,105000.0,125000.0,16.0,15,4.0,2,https://tiki.vn/but-long-to-mau-crylic-marker-...,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tiki
6,137161811,Bút gel bấm ngòi 0.5mm nhiều màu Deli - Độ dài...,9000.0,16200.0,44.0,9783,5.0,821,https://tiki.vn/but-gel-bam-0-5mm-nhieu-mau-de...,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tiki
11,105614684,Bút gel 0.5mm Deli nhiều màu lựa chọn - Bút bi...,7000.0,12600.0,44.0,13719,4.8,503,https://tiki.vn/but-gel-0-5mm-deli-nhieu-mau-l...,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tiki
12,105614665,Bút bi bấm ngòi 0.7mm mực xanh Deli - Màu sắc ...,136200.0,151200.0,10.0,2763,4.8,233,https://tiki.vn/but-bi-bam-ngoi-0-7mm-muc-xanh...,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tiki
13,252041787,Bút Bi Nước Mực Gel Màu Đen Cao Cấp Ngòi 0.5mm...,10000.0,18000.0,44.0,31,NaN,0,https://tiki.vn/but-bi-gel-dau-bam-muc-den-del...,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tiki
16,95090804,Bút dạ viết bảng Deli - xóa được - Mực xanh/đe...,24000.0,43200.0,44.0,4712,5.0,270,https://tiki.vn/but-da-viet-bang-deli-xoa-duoc...,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tiki
19,259892096,Bút Viết Dạ Bảng Bút Lông Xoá Được 8 Màu Kèm B...,13000.0,23400.0,44.0,164,5.0,4,https://tiki.vn/set-8-but-viet-da-bang-but-lon...,https://salt.tikicdn.com/cache/280x280/ts/prod...,Tiki


In [7]:
import unicodedata
import re

def clean_string(text):
    text = unicodedata.normalize('NFD', text)   # Chuẩn hóa Unicode và loại bỏ dấu tiếng Việt
    text = ''.join([char for char in text if unicodedata.category(char) != 'Mn'])
    text = re.sub(r'[^a-zA-Z0-9]', '', text)    # Loại bỏ ký tự đặc biệt và khoảng trắng
    
    return text.lower()# Chuyển về chữ thường


In [ ]:
# df.reset_index(drop=True, inplace=True)

In [8]:
x = [clean_string(word) for word in keyword.split()]
y = []

x

for item in df['ProductName']:
    y.append(clean_string(item))

# #------------> for item, index in zip(y, range(len(y))):
# #------------>     if x not in item:
# #------------>         df.drop(index = index, inplace = True)
indexes_to_drop = [i for i, item in enumerate(y) if not all(word in item for word in x)]
df.drop(index=indexes_to_drop, inplace=True)
# df.reset_index(drop=True, inplace=True)

In [ ]:
# df.sort_values(by='Price', inplace=True)


In [ ]:
# import mysql.connector

# config = {
#     "host": "localhost",
#     "user": "root",
#     "password": "111111111",
#     "database": "ecommerce_db"
# }

In [ ]:
# def get_db_connection():
#     conn = mysql.connector.connect(**config)
#     return conn


In [ ]:
# def Get_Data_Tiki(df):
#     connection = get_db_connection()
#     cursor = connection.cursor()

#     query = """
#             INSERT INTO products (source_productID, ProductName, price, original_price, discount, quantity_sold, rating, review_count, url_image, url_product, source)
#             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
#             """

#     for _, row in df.iterrows():
#         cursor.execute(query, (
#             row['Source_productID'], row['ProductName'], row['Price'], row['Original_Price'], row['Discount'],
#             row['Quantity_Sold'], row['Rating'], row['Review_Count'], row['URL_Image'],
#             row['URL_Product'], row['Source']
#         ))

#     connection.commit()
#     cursor.close()
#     connection.close()


In [ ]:
# df.head(20)

In [ ]:
# Get_Data_Tiki(df)